In [1]:
!pip install wikiextractor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 1.3 MB/s eta 0:00:00


In [16]:
!wget https://dumps.wikimedia.org/viwiki/20240701/viwiki-20240701-pages-articles-multistream5.xml-p9680045p11180044.bz2

--2024-07-16 16:01:44--  https://dumps.wikimedia.org/viwiki/20240701/viwiki-20240701-pages-articles-multistream5.xml-p9680045p11180044.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3219583 (3.1M) [application/octet-stream]
Saving to: ‘viwiki-20240701-pages-articles-multistream5.xml-p9680045p11180044.bz2’

viwiki-20240701-pag 100%[===================>]   3.07M  3.91MB/s    in 0.8s    

2024-07-16 16:01:45 (3.91 MB/s) - ‘viwiki-20240701-pages-articles-multistream5.xml-p9680045p11180044.bz2’ saved [3219583/3219583]



In [17]:
!python -m wikiextractor.WikiExtractor '/content/viwiki-20240701-pages-articles-multistream5.xml-p9680045p11180044.bz2'

INFO: Preprocessing '/content/viwiki-20240701-pages-articles-multistream5.xml-p9680045p11180044.bz2' to collect template definitions: this may take some time.
INFO: Loaded 171 templates in 1.1s
INFO: Starting page extraction from /content/viwiki-20240701-pages-articles-multistream5.xml-p9680045p11180044.bz2.
INFO: Using 1 extract processes.
INFO: Finished 1-process extraction of 1931 articles in 3.6s (543.6 art/s)


In [19]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00


In [24]:
from pyvi.ViTokenizer import tokenize
import re, os, string
import pandas as pd

def clean_text(text):
    text = re.sub('<.*?>', '', text).strip()
    text = re.sub('(\s)+', r'\1', text)
    return text

def normalize_text(text):
    listpunctuation = string.punctuation.replace('_', '')
    for i in listpunctuation:
        text = text.replace(i, ' ')
    return text.lower()

# list stopwords
filename = './stopwords.csv'
data = pd.read_csv(filename, sep="\t", encoding='utf-8')
list_stopwords = data['stopwords']

def remove_stopword(text):
    pre_text = []
    words = text.split()
    for word in words:
        if word not in list_stopwords:
            pre_text.append(word)
    text2 = ' '.join(pre_text)

    return text2

def sentence_segment(text):
    sents = re.split("([.?!])?[\n]+|[.?!] ", text)
    return sents


def word_segment(sent):
    sent = tokenize(sent)
    return sent

path_to_corpus = '/content/text'


f_w = open('./datatrain.txt', 'w')
for i, sub_dir in enumerate(os.listdir(path_to_corpus)):
    path_to_subdir = path_to_corpus + '/' + sub_dir
    for j, file_name in enumerate(os.listdir(path_to_subdir)):
        with open(path_to_subdir + '/' + file_name) as f_r:
            contents = f_r.read().strip().split('</doc>')
            for content in contents:
                if (len(content) < 5):
                    continue
                content = clean_text(content)
                sents = sentence_segment(content)
                for sent in sents:
                    if(sent != None):
                        sent = word_segment(sent)
                        sent = remove_stopword(normalize_text(sent))
                        if(len(sent.split()) > 1):
                            f_w.write(sent + '\n')
            print(f'Done , {i + 1}, :, {j + 1}')

f_w.close()

Done , 1, :, 1
Done , 1, :, 2
Done , 1, :, 3
Done , 1, :, 4
Done , 1, :, 5
